**Demo for `teneva.core.svd`**

---

Module contains the basic implementation of the TT-SVD algorithm as well as new original TT-SVD-incomplete algorithm, which implements efficient construction of the TT-tensor based on specially selected elements. This module also contains functions for constructing the SVD decomposition and skeleton decomposition for the matrices.

## Loading and importing modules

In [1]:
import numpy as np
import teneva
from time import perf_counter as tpc
np.random.seed(42)

## Function `matrix_skeleton`

Construct truncated skeleton decomposition A = U V for the given matrix.

In [2]:
m, n = 100, 30                                        # Shape of the matrix
A = np.outer(np.random.randn(m), np.random.randn(n))  # Build random matrix,
A += np.outer(np.random.randn(m), np.random.randn(n)) # which has rank 3,
A += np.outer(np.random.randn(m), np.random.randn(n)) # as a sum of rank-1 matrices

In [3]:
U, V = teneva.matrix_skeleton(A, e=1.E-10)            # Compute skeleton decomp.
e = np.linalg.norm(A - U @ V) / np.linalg.norm(A)     # Approximation error
print(f'Shape of U :', U.shape)
print(f'Shape of V :',V.shape)
print(f'Error      : {e:-8.2e}')

Shape of U : (100, 3)
Shape of V : (3, 30)
Error      : 6.09e-16


In [4]:
U, V = teneva.matrix_skeleton(A, r=2)                 # Compute skeleton decomp with small rank
e = np.linalg.norm(A - U @ V) / np.linalg.norm(A)     # Approximation error
print(f'Shape of U :', U.shape)
print(f'Shape of V :',V.shape)
print(f'Error      : {e:-8.2e}')

Shape of U : (100, 2)
Shape of V : (2, 30)
Error      : 4.60e-01


## Function `matrix_svd`

Construct truncated SVD decomposition A = U V for the given matrix.

In [5]:
m, n = 100, 30                                        # Shape of the matrix
A = np.outer(np.random.randn(m), np.random.randn(n))  # Build random matrix,
A += np.outer(np.random.randn(m), np.random.randn(n)) # which has rank 3,
A += np.outer(np.random.randn(m), np.random.randn(n)) # as a sum of rank-1 matrices

In [6]:
U, V = teneva.matrix_svd(A, e=1.E-10)                 # Compute SVD-decomp.
e = np.linalg.norm(A - U @ V) / np.linalg.norm(A)     # Approximation error
print(f'Shape of U :', U.shape)
print(f'Shape of V :',V.shape)
print(f'Error      : {e:-8.2e}')

Shape of U : (100, 17)
Shape of V : (17, 30)
Error      : 5.00e-16


In [7]:
U, V = teneva.matrix_svd(A, r=3)                      # Compute SVD-decomp.
e = np.linalg.norm(A - U @ V) / np.linalg.norm(A)     # Approximation error
print(f'Shape of U :', U.shape)
print(f'Shape of V :',V.shape)
print(f'Error      : {e:-8.2e}')

Shape of U : (100, 3)
Shape of V : (3, 30)
Error      : 4.70e-16


In [8]:
U, V = teneva.matrix_svd(A, e=1.E-2)                  # Compute SVD-decomp.
e = np.linalg.norm(A - U @ V) / np.linalg.norm(A)     # Approximation error
print(f'Shape of U :', U.shape)
print(f'Shape of V :',V.shape)
print(f'Error      : {e:-8.2e}')

Shape of U : (100, 3)
Shape of V : (3, 30)
Error      : 4.70e-16


In [9]:
U, V = teneva.matrix_svd(A, r=2)                      # Compute SVD-decomp.
e = np.linalg.norm(A - U @ V) / np.linalg.norm(A)     # Approximation error
print(f'Shape of U :', U.shape)
print(f'Shape of V :',V.shape)
print(f'Error      : {e:-8.2e}')

Shape of U : (100, 2)
Shape of V : (2, 30)
Error      : 4.37e-01


## Function `svd`

Construct TT-tensor from the given full tensor using TT-SVD algorithm.

In [10]:
d = 20                                          # Dimension number
t = np.arange(2**d)                             # Tensor will be 2^d
Z_full = np.cos(t).reshape([2] * d, order='F')  # Construct d-dim full array

In [11]:
Y = teneva.svd(Z_full)                          # Construct TT-tensor by TT-SVD
Y_full = teneva.full(Y)                         # Convert it back to numpy to check result
e = np.linalg.norm(Y_full - Z_full)             # Compute error for TT-tensor vs full tensor 
e /= np.linalg.norm(Z_full)                     #

In [12]:
print(f'Size (np) : {Z_full.size:-8d}')         # Size of original tensor
print(f'Size (tt) : {teneva.size(Y):-8d}')      # Size of the TT-tensor
print(f'Erank     : {teneva.erank(Y):-8.2f}')   # Eff. rank of the TT-tensor
print(f'Error     : {e:-8.2e}')                 # Rel. error for TT-tensor vs full tensor

Size (np) :  1048576
Size (tt) :      152
Erank     :     2.00
Error     : 1.58e-14


## Function `svd_incomplete`

Construct TT-tensor from the given specially selected samples.

In [13]:
d = 20                                              # Dimension number
n = [2] * d                                         # Shape of the tensor/grid
t = np.arange(2**d)                                 # Tensor will be 2^d
Z_full = np.cos(t).reshape([2] * d, order='F')      # Construct d-dim full array

In [14]:
m = 4                                               # The expected TT-rank
I_trn, idx, idx_many = teneva.sample_tt(n, m)       # Generate special samples (indices) for the tensor

In [15]:
Y_trn = np.array([Z_full[tuple(i)] for i in I_trn]) # Compute tensor values in I multiindices

In [16]:
Y = teneva.svd_incomplete(I_trn, Y_trn,
    idx, idx_many, e=1.E-10, r=3)                   # Construct TT-tensor
teneva.show(Y)                                      # Show the tensor

  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2 
 / \/ \/ \/ \/ \/ \/ \/ \/ \/ \/ \/ \/ \/ \/ \/ \/ \/ \/ \/ \
 1  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  1 



In [17]:
Y_full = teneva.full(Y)                          # Convert it back to numpy to check result
e = np.linalg.norm(Y_full - Z_full)              # Compute error for TT-tensor vs full tensor 
e /= np.linalg.norm(Z_full)                      #

In [18]:
print(f'Size (np) : {Z_full.size:-8d}')          # Size of original tensor
print(f'Size (tt) : {teneva.size(Y):-8d}')       # Size of the TT-tensor
print(f'Erank     : {teneva.erank(Y):-8.2f}')    # Eff. rank of the TT-tensor
print(f'Error     : {e:-8.2e}')                  # Rel. error for TT-tensor vs full tensor

Size (np) :  1048576
Size (tt) :      152
Erank     :     2.00
Error     : 2.01e-15


---